## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-22-14-53-41 +0000,nypost,Hollywood actors’ union secretly plotting poss...,https://nypost.com/2025/12/22/media/hollywood-...
1,2025-12-22-14-53-00 +0000,wsj,How to Invest in Gold,https://www.wsj.com/finance/investing/buy-gold...
2,2025-12-22-14-52-00 +0000,wsj,CBS News Editor Bari Weiss Defends Decision to...,https://www.wsj.com/business/media/cbs-news-ed...
3,2025-12-22-14-51-10 +0000,wapo,‘60 Minutes’ correspondent says CBS’s Bari Wei...,https://www.washingtonpost.com/business/2025/1...
4,2025-12-22-14-49-17 +0000,nypost,Jimmy Kimmel to make Christmas Day address on ...,https://nypost.com/2025/12/22/media/jimmy-kimm...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2370/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,24
192,up,11
519,was,10
211,epstein,9
321,bondi,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
200,2025-12-21-20-20-00 +0000,wsj,"Before Charlie Kirk was fatally shot, he was r...",https://www.wsj.com/politics/policy/charlie-ki...,62
66,2025-12-22-11-29-46 +0000,wapo,Car bomb in Moscow kills senior Russian genera...,https://www.washingtonpost.com/world/2025/12/2...,56
211,2025-12-21-19-18-08 +0000,nypost,Nicki Minaj makes surprise appearance with Eri...,https://nypost.com/2025/12/21/us-news/nicki-mi...,52
215,2025-12-21-18-40-44 +0000,nyt,Epstein Files Photos Disappear From Government...,https://www.nytimes.com/2025/12/20/us/politics...,52
203,2025-12-21-20-10-45 +0000,nypost,Trump’s No. 2 at DOJ Todd Blanche emphatic Eps...,https://nypost.com/2025/12/21/us-news/trumps-n...,51


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
200,62,2025-12-21-20-20-00 +0000,wsj,"Before Charlie Kirk was fatally shot, he was r...",https://www.wsj.com/politics/policy/charlie-ki...
66,56,2025-12-22-11-29-46 +0000,wapo,Car bomb in Moscow kills senior Russian genera...,https://www.washingtonpost.com/world/2025/12/2...
215,52,2025-12-21-18-40-44 +0000,nyt,Epstein Files Photos Disappear From Government...,https://www.nytimes.com/2025/12/20/us/politics...
225,40,2025-12-21-17-30-34 +0000,nypost,US military targeting third tanker off coast o...,https://nypost.com/2025/12/21/world-news/us-mi...
220,37,2025-12-21-17-57-50 +0000,latimes,'Avatar: Fire and Ash' heats up the box office...,https://www.latimes.com/entertainment-arts/bus...
128,34,2025-12-22-06-14-56 +0000,bbc,Australian move to fast-track new gun and prot...,https://www.bbc.com/news/articles/cj9ydj7y0vvo...
205,33,2025-12-21-19-56-00 +0000,wsj,A top Justice Department official said Sunday ...,https://www.wsj.com/politics/policy/justice-de...
129,30,2025-12-22-06-01-44 +0000,latimes,CBS News correspondent accuses Bari Weiss of '...,https://www.latimes.com/entertainment-arts/bus...
151,30,2025-12-22-02-15-24 +0000,nyt,Waymo Suspended Service in San Francisco After...,https://www.nytimes.com/2025/12/21/us/waymo-su...
230,25,2025-12-21-16-08-05 +0000,nypost,Russia claims talks on US peace plan for Ukrai...,https://nypost.com/2025/12/21/world-news/russi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
